## Global warming CDD Levels

In [1]:
# !pip install xlrd

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import pandas as pd
import os
# import sys
import xarray as xr
import xclim

/home/ubuntu/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [3]:
## Import all RCP8.5 files 

In [4]:
os.chdir('/mys3bucket/Biascorrected/') # Setting working directory

In [5]:
from glob import glob

In [6]:
## Get all bias corrected under RCP85 files
files_bias=glob('/mys3bucket/Biascorrected/tas_day_BCSD_rcp85*_Bias-eqm.nc') # list of cordex

In [7]:
files_bias

['/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_CNRM-CM5_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_CSIRO-Mk3-6-0_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_GFDL-ESM2M_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_IPSL-CM5A-MR_1991_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_IPSL-CM5A-MR_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_MIROC5_2006_2099_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_MPI-ESM-LR_1991_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_MPI-ESM-LR_2006_2100_WAfrik_rg_Bias-eqm.nc',
 '/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_NorESM1-M_2006_2100_WAfrik_rg_Bias-eqm.nc']

In [8]:
GWL_tab=pd.read_excel('/mys3bucket/NEX_GDDP_Global_warming_levels.xlsx',engine='openpyxl')

In [9]:
GWL_tab

,NEX-GDDP Models,Resolution,Experiment,warming_level,start_year,end_year
0,CNRM-CM5,22 km,RCP85,1.5,2015,2044
1,CSIRO-Mk3-6-0,22 km,RCP85,1.5,2018,2047
2,IPSL-CM5A-MR,22 km,RCP85,1.5,2002,2031
3,MIROC5,22 km,RCP85,1.5,2019,2048
4,MPI-ESM-LR,22 km,RCP85,1.5,2004,2033
5,NorESM1-M,22 km,RCP85,1.5,2019,2048
6,GFDL-ESM2M,22 km,RCP85,1.5,2020,2049
7,CNRM-CM5,22 km,RCP85,2.0,2029,2058
8,CNRM-CM5,22 km,RCP85,2.5,2041,2070
9,CNRM-CM5,22 km,RCP85,3.0,2052,2081


In [10]:
start_year=GWL_tab[' start_year'][0]
end_year=GWL_tab[' end_year'][0]
GWL_tab.columns

Index(['NEX-GDDP Models', 'Resolution', 'Experiment', ' warming_level',
       ' start_year', ' end_year'],
      dtype='object')

In [11]:
def bias_process_gwl(start_year,end_year,target_file,model_name,scenario):
       
    print (target_file[0])
    
    dat_bias=xr.open_dataarray(target_file[0])
    
    # Set unit back again to Kelvin for the package xclim
    dat_bias=dat_bias+273.15
    dat_bias.attrs["units"] = "K"
    
    print (start_year)
    print(end_year)
    dat_bias=dat_bias.sel(time=slice(str(start_year)+"-01-01", str(end_year)+"-12-31"))
    print ('Pass open file')
    print (dat_bias)
    cdd = xclim.atmos.cooling_degree_days(tas=dat_bias, thresh="25.0 degC", freq="YS")
    
    # Get output file_name
    split_name=target_file[0].split('.')
    outputfile='/mys3bucket/Biascorrected/cdd_output/CDD/CDD_GWL_'+str(scenario)+'_tas_BCSD_rcp85_r1i1p1_'+str(model_name)+'_'+str(start_year)+'_'+str(end_year)+'_WAfrik_rg_Bias-eqm.nc'                 
    
    
    # Create outpuut file
    print (outputfile)
    
    cdd.to_netcdf(outputfile)

In [35]:
from netCDF4 import Dataset
import h5py

In [13]:
# cdd = xclim.atmos.cooling_degree_days(tas=dat_bias, thresh="25.0 degC", freq="YS")

In [15]:
for i in range(GWL_tab.shape[0]):
    print(GWL_tab['NEX-GDDP Models'][i])
    #Get the right file to be processed
    target_file=glob('/mys3bucket/Biascorrected/tas_day_BCSD_rcp85_r1i1p1_'+str(GWL_tab['NEX-GDDP Models'][i])+'*_Bias-eqm.nc')
    
    # Get the start and the end year
    start_year=GWL_tab[' start_year'][i]
    end_year=GWL_tab[' end_year'][i]
    scenario=GWL_tab[' warming_level'][i]
    
    print ('Print target file')
    print(target_file)
    
    bias_process_gwl(start_year,end_year,target_file,GWL_tab['NEX-GDDP Models'][i],scenario)

## Computation Historical CDD for the ERA Data

In [16]:
# MME_ERA=xr.open_dataset('/mys3bucket/ERA_Temp/Daily_air_temperature_at_2_metres_1970_2020_WA.nc')
# MME_ERA=MME_ERA.drop(['time_bnds'])
# MME_ERA_df=MME_ERA.t2m-273.15
# # copy attributes to get nice figure labels and change Kelvin to Celsius
# MME_ERA_df.attrs = MME_ERA.t2m.attrs
# MME_ERA_df.attrs["units"] = "deg C"
# MME_ERA_df=MME_ERA_df.rename({'latitude': 'lat','longitude': 'lon'})

# #Time slice from 1970 TO 2000
# obs=MME_ERA_df.sel(time=slice("1970-01-01", "2000-12-31"))
# # Resample to daily the time series
# daily_ds = obs.resample(time="D").mean(keep_attrs=True)


# # Set unit back again to Kelvin for the package xclim
# daily_ds=daily_ds+273.15
# daily_ds.attrs["units"] = "K"

# # Run CDD script
# cdd_obs=xclim.atmos.cooling_degree_days(tas=daily_ds, thresh="25.0 degC", freq="YS")
# cdd_obs.to_netcdf('/mys3bucket/Biascorrected/cdd_output/CDD/CDD_Daily_air_temperature_at_2_metres_1970_2000_WA.nc')

In [17]:
MME_ERA=xr.open_dataset('/mys3bucket/ERA_Temp/Daily_air_temperature_at_2_metres_1970_2020_WA.nc')
MME_ERA=MME_ERA.drop(['time_bnds'])
MME_ERA_df=MME_ERA.t2m-273.15
# copy attributes to get nice figure labels and change Kelvin to Celsius
MME_ERA_df.attrs = MME_ERA.t2m.attrs
MME_ERA_df.attrs["units"] = "deg C"
MME_ERA_df=MME_ERA_df.rename({'latitude': 'lat','longitude': 'lon'})

#Time slice from 1971 TO 2000
obs=MME_ERA_df.sel(time=slice("1971-01-01", "2000-12-31"))
# Resample to daily the time series
daily_ds = obs.resample(time="D").mean(keep_attrs=True)


# Set unit back again to Kelvin for the package xclim
daily_ds=daily_ds+273.15
daily_ds.attrs["units"] = "K"

# Run CDD script
cdd_obs=xclim.atmos.cooling_degree_days(tas=daily_ds, thresh="25.0 degC", freq="YS")
cdd_obs.to_netcdf('/mys3bucket/cdd_output/CDD/CDD_Daily_air_temperature_at_2_metres_1971_2000_WA.nc')

/home/ubuntu/.local/lib/python3.6/site-packages/xarray/coding/times.py:134: CFWarning: this date/calendar/year zero convention is not supported by CF
  cftime.num2date(num_dates, units, calendar, only_use_cftime_datetimes=True)
/home/ubuntu/.local/lib/python3.6/site-packages/xarray/coding/times.py:463: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ubuntu/.local/lib/python3.6/site-packages/xclim/indicators/atmos/_temperature.py:72: UserWarning: Variable does not have a `cell_methods` attribute.
  cfchecks.check_valid(tas, "cell_methods", "*time: mean within days*")
/home/ubuntu/.local/lib/python3.6/site-packages/xclim/indicators/atmos/_temperature.py:73: UserWarning: Variable does not have a `standard_name` attribute.
  cfchecks.check_valid(tas, "standard_name", "air_temperature")


In [19]:
# dd=xr.open_dataset('/mys3bucket/cdd_output/CDD/CDD_GWL_3.0_tas_BCSD_rcp85_r1i1p1_GFDL-ESM2M_2066_2095_WAfrik_rg_Bias-eqm.nc')

In [20]:
# dd.cooling_degree_days.plot()

# Compute CDD by models and Multi-Model Mean for the historical period

In [21]:
# os.chdir('/var/s3/Hist/WestAfrica/Agg/') # Setting working directory

In [22]:
## Get all historical data

In [23]:
all_hist_files=glob('/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical*.nc') # list of historical files
all_hist_files

['/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_CNRM-CM5_1950_2005_WAfrik_rg.nc',
 '/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1950_2005_WAfrik_rg.nc',
 '/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_GFDL-ESM2M_1950_2005_WAfrik_rg.nc',
 '/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1950_2005_WAfrik_rg.nc',
 '/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_MIROC5_1950_2005_WAfrik_rg.nc',
 '/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1950_2005_WAfrik_rg.nc',
 '/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_NorESM1-M_1950_2005_WAfrik_rg.nc']

In [25]:
# tes=xr.open_dataset('/var/s3/Hist/WestAfrica/Agg/tas_day_BCSD_historical_r1i1p1_NorESM1-M_1950_2005_WAfrik.nc')
# tes

In [26]:
#function that run the cdd through all files 
def cdd_process_hist_files(start_year,end_year,file_path,model_name):
#     print (file_path)
    
    dat_bias=xr.open_dataset(file_path)
    dat_bias_df=dat_bias.tasmax
    dat_bias_df.attrs = dat_bias.tasmax.attrs
    
    df=dat_bias_df.sel(time=slice(str(start_year)+"-01-01", str(end_year)+"-12-31"))
    
    cdd = xclim.atmos.cooling_degree_days(tas=df, thresh="25.0 degC", freq="YS")
    
#     split_name=file.split('.')
    outputfile='/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_'+str(model_name)+'_'+str(start_year)+'_'+str(end_year)+'_WAfrik.nc'
    
    print (outputfile)
    cdd.to_netcdf(outputfile)

In [27]:
NEX_name=['CNRM-CM5','CSIRO-Mk3-6-0','GFDL-ESM2M_','IPSL-CM5A-MR','MIROC5','MPI-ESM-LR','NorESM1-M'] # Models prefix labels

In [28]:
for i,model_name in enumerate(NEX_name):
    print(all_hist_files[i])
    print(model_name)
    cdd_process_hist_files(1971,2000,all_hist_files[i],model_name)

/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_CNRM-CM5_1950_2005_WAfrik_rg.nc
CNRM-CM5


/home/ubuntu/.local/lib/python3.6/site-packages/xclim/indicators/atmos/_temperature.py:72: UserWarning: Variable has a non-conforming cell_methods. Got `time: maximum`, expected `['*time: mean within days*']`
  cfchecks.check_valid(tas, "cell_methods", "*time: mean within days*")


/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_CNRM-CM5_1971_2000_WAfrik.nc
/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1950_2005_WAfrik_rg.nc
CSIRO-Mk3-6-0


/home/ubuntu/.local/lib/python3.6/site-packages/xclim/indicators/atmos/_temperature.py:72: UserWarning: Variable has a non-conforming cell_methods. Got `time: maximum`, expected `['*time: mean within days*']`
  cfchecks.check_valid(tas, "cell_methods", "*time: mean within days*")


/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1971_2000_WAfrik.nc
/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_GFDL-ESM2M_1950_2005_WAfrik_rg.nc
GFDL-ESM2M_


...

/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_MIROC5_1971_2000_WAfrik.nc
/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1950_2005_WAfrik_rg.nc
MPI-ESM-LR


/home/ubuntu/.local/lib/python3.6/site-packages/xclim/indicators/atmos/_temperature.py:72: UserWarning: Variable has a non-conforming cell_methods. Got `time: maximum`, expected `['*time: mean within days*']`
  cfchecks.check_valid(tas, "cell_methods", "*time: mean within days*")


/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1971_2000_WAfrik.nc
/mys3bucket/NEX_FUT_DAT/tas_day_BCSD_historical_r1i1p1_NorESM1-M_1950_2005_WAfrik_rg.nc
NorESM1-M


/home/ubuntu/.local/lib/python3.6/site-packages/xclim/indicators/atmos/_temperature.py:72: UserWarning: Variable has a non-conforming cell_methods. Got `time: maximum`, expected `['*time: mean within days*']`
  cfchecks.check_valid(tas, "cell_methods", "*time: mean within days*")


/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_NorESM1-M_1971_2000_WAfrik.nc


In [41]:
from cdo import *
cdo = Cdo()     #-- make it easier

In [42]:
xr.open_dataset('/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_CNRM-CM5_1971_2000_WAfrik.nc')

<xarray.Dataset>
Dimensions:              (lat: 121, lon: 181, time: 30)
Coordinates:
  * time                 (time) datetime64[ns] 1971-01-01 ... 2000-01-01
  * lon                  (lon) float32 -20.0 -19.75 -19.5 ... 24.5 24.75 25.0
  * lat                  (lat) float32 30.0 29.75 29.5 29.25 ... 0.5 0.25 0.0
Data variables:
    cooling_degree_days  (time, lat, lon) float32 ...

In [45]:
from cdo import *
cdo = Cdo()     #-- make it easier

#Compute Historical ensemble mean
cdo.ensmean(input='/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical_r1i1p1_*.nc',
             output='/mys3bucket/cdd_output/CDD/MMM_CDD_tas_day_BCSD_historical_r1i1p1_1971_2000.nc')

cdo.timmean(input='/mys3bucket/cdd_output/CDD/MMM_CDD_tas_day_BCSD_historical_r1i1p1_1971_2000.nc',
             output='/mys3bucket/cdd_output/CDD/CLIM_MMM_CDD_tas_day_BCSD_historical_r1i1p1_1971_2000.nc')

In [115]:
obs=xr.open_dataset('/mys3bucket/cdd_output/CDD/CDD_Daily_air_temperature_at_2_metres_1970_2000_WA.nc')
obs_df=obs.sel(time=slice("1971-01-01", "2000-12-31")).mean('time')
obs_df.to_netcdf('/mys3bucket/cdd_output/CDD/CLIM_CDD_Daily_air_temperature_at_2_metres_1970_2000_WA.nc')

In [114]:
obs_df

<xarray.Dataset>
Dimensions:              (lon: 181, lat: 121)
Coordinates:
  * lon                  (lon) float32 -20.0 -19.75 -19.5 ... 24.5 24.75 25.0
  * lat                  (lat) float32 30.0 29.75 29.5 29.25 ... 0.5 0.25 0.0
Data variables:
    cooling_degree_days  (lat, lon) float32 0.03804 0.03287 ... 31.88 32.04

In [ ]:
cdo.timmean(input='/mys3bucket/cdd_output/CDD/MMM_CDD_tas_day_BCSD_historical_r1i1p1_1971_2000.nc',
             output='/mys3bucket/cdd_output/CDD/CLIM_MMM_CDD_tas_day_BCSD_historical_r1i1p1_1971_2000.nc')

In [102]:
from glob import glob

In [103]:
# Climatological mean of each model
all_hist_files=glob('/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_historical*.nc') # list of historical files
all_hist_files

['/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_CNRM-CM5_1971_2000_WAfrik.nc',
 '/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1971_2000_WAfrik.nc',
 '/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_GFDL-ESM2M__1971_2000_WAfrik.nc',
 '/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1971_2000_WAfrik.nc',
 '/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_MIROC5_1971_2000_WAfrik.nc',
 '/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1971_2000_WAfrik.nc',
 '/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_NorESM1-M_1971_2000_WAfrik.nc']

In [116]:
## MME of HISTORICAL CDD

In [105]:
for i,model_name in enumerate(NEX_name):
    
    outputfile='/mys3bucket/cdd_output/CDD/CLIM_'+all_hist_files[i].split('/')[3]
    print(all_hist_files[i])
    cdo.timmean(input=all_hist_files[i],output=outputfile)

/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_CNRM-CM5_1971_2000_WAfrik.nc
/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1971_2000_WAfrik.nc
/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_GFDL-ESM2M__1971_2000_WAfrik.nc
/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1971_2000_WAfrik.nc
/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_MIROC5_1971_2000_WAfrik.nc
/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1971_2000_WAfrik.nc
/mys3bucket/Historical_cdd/CDD_tas_day_BCSD_historical_r1i1p1_NorESM1-M_1971_2000_WAfrik.nc


## Compute Multi-Model Mean CDD under RCP 4.5 and RCP 8.5

In [17]:
from cdo import *
cdo = Cdo()     #-- make it easier

In [19]:
# glob('/mys3bucket/Biascorrected/CDD_tas_day_BCSD_rcp85*_2006_*_WAfrik_rg*.nc')

In [20]:
## All RCP MODELS ENSMEAN RCP 8.5
cdo.ensmean(input='/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_rcp85*_2006_*_WAfrik_rg*.nc',
             output='/mys3bucket/cdd_output/CDD/MMM_CDD_tas_day_BCSD_rcp85_r1i1p1_2006_2100_WAfrik_rg_Bias-eqm.nc')

'/mys3bucket/Biascorrected/MMM_CDD_tas_day_BCSD_rcp85_r1i1p1_2006_2100_WAfrik_rg_Bias-eqm.nc'

In [35]:
dd=xr.open_dataset('/mys3bucket/cdd_output/CDD/MMM_CDD_tas_day_BCSD_rcp85_r1i1p1_2006_2100_WAfrik_rg_Bias-eqm.nc')
# dd.cooling_degree_days.plot()

In [36]:
dd.cooling_degree_days.mean(['lon','lat']).to_dataframe()

,cooling_degree_days
time,
2006-01-01,902.041046
2007-01-01,938.115071
2008-01-01,947.308226
2009-01-01,975.842594
2010-01-01,978.538826
...,...
2095-01-01,2237.120379
2096-01-01,2275.141584
2097-01-01,2302.885337


In [22]:
## All RCP MODELS ENSMEAN RCP 4.5
cdo.ensmean(input='/mys3bucket/cdd_output/CDD/CDD_tas_day_BCSD_rcp45*.nc',
             output='/mys3bucket/cdd_output/CDD/MMM_CDD_tas_day_BCSD_rcp45_r1i1p1_2006_2100_WAfrik_rg_Bias-eqm.nc')

'/mys3bucket/Biascorrected/MMM_CDD_tas_day_BCSD_rcp45_r1i1p1_2006_2100_WAfrik_rg_Bias-eqm.nc'

In [46]:
cdd_rcp45=xr.open_dataset('/mys3bucket/cdd_output/CDD/MMM_CDD_tas_day_BCSD_rcp45_r1i1p1_2006_2100_WAfrik_rg_Bias-eqm.nc')
# cdd_rcp45.cooling_degree_days.plot()

In [25]:
cdd_rcp45.cooling_degree_days.mean(['lon','lat']).to_dataframe().head()

,cooling_degree_days
time,
2006-01-01,911.701219
2007-01-01,888.731613
2008-01-01,924.879191
2009-01-01,946.217493
2010-01-01,969.543201
...,...
2095-01-01,1448.614589
2096-01-01,1529.281029
2097-01-01,1477.828347


## Compute Multi-Model Mean CDD under Global Warming Levels under RCP8.5

In [142]:
# from cdo import *
# cdo = Cdo()     #-- make it easier

In [26]:
## All RCP MODELS ENSMEAN GWL 1.0 under RCP8.5
cdo.ensmean(input='/mys3bucket/cdd_output/CDD/CDD_GWL_1.5_tas_BCSD_rcp85*.nc',
             output='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_1.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

cdo.timmean(input='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_1.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc',
             output='/mys3bucket/cdd_output/CDD/CLIM_MMM_CDD_GWL_1.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

'/mys3bucket/Biascorrected/CLIM_MMM_CDD_GWL_1.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc'

In [27]:
## All RCP MODELS ENSMEAN GWL 2 under RCP8.5
cdo.ensmean(input='/mys3bucket/cdd_output/CDD/CDD_GWL_2.0_tas_BCSD_rcp85*.nc',
             output='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_2.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

cdo.timmean(input='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_2.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc',
             output='/mys3bucket/cdd_output/CDD/CLIM_MMM_CDD_GWL_2.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

'/mys3bucket/Biascorrected/CLIM_MMM_CDD_GWL_2.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc'

In [28]:
## All RCP MODELS ENSMEAN GWL 2.5 under RCP8.5
cdo.ensmean(input='/mys3bucket/cdd_output/CDD/CDD_GWL_2.5_tas_BCSD_rcp85*.nc',
             output='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_2.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

cdo.timmean(input='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_2.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc',
             output='/mys3bucket/cdd_output/CDD/CLIM_MMM_CDD_GWL_2.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

'/mys3bucket/Biascorrected/CLIM_MMM_CDD_GWL_2.5_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc'

In [29]:
## All RCP MODELS ENSMEAN GWL 3 under RCP8.5
cdo.ensmean(input='/mys3bucket/cdd_output/CDD/CDD_GWL_3.0_tas_BCSD_rcp85*.nc',
             output='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_3.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

cdo.timmean(input='/mys3bucket/cdd_output/CDD/MMM_CDD_GWL_3.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc',
             output='/mys3bucket/cdd_output/CDD/CLIM_MMM_CDD_GWL_3.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc')

'/mys3bucket/Biascorrected/CLIM_MMM_CDD_GWL_3.0_tas_BCSD_rcp85_r1i1p1_WAfrik_rg_Bias-eqm.nc'

## Compute population global warming levels

In [35]:
# import population data
pop=xr.open_dataset('/mys3bucket/Pop_ssp5/ssp5_total_2000_2100_WA_rg_set.nc')
## Assign proper time indexes
pop['time']=pd.date_range("2000-01-01", freq="Y", periods=101)

In [37]:
pop.sel(time=slice(str(2010)+"-01-01", str(2020)+"-12-31"))

<xarray.Dataset>
Dimensions:  (time: 11, lon: 181, lat: 121)
Coordinates:
  * time     (time) datetime64[ns] 2010-12-31 2011-12-31 ... 2020-12-31
  * lon      (lon) float32 -20.0 -19.75 -19.5 -19.25 ... 24.25 24.5 24.75 25.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 1.0 0.75 0.5 0.25 0.0
Data variables:
    Band1    (time, lat, lon) float32 ...
Attributes: (12/13)
    CDI:                 Climate Data Interface version ?? (http://mpimet.mpg...
    history:             Sun Jan 30 12:42:06 2022: cdo mergetime baseYr_total...
    Conventions:         CF-1.5
    GDAL:                GDAL 2.1.3, released 2017/20/01
    GDAL_DataType:       Generic
    GDAL_AREA_OR_POINT:  Area
    ...                  ...
    NCL_Version:         6.5.0
    system:              Linux cisl-abbott 3.10.0-693.5.2.el7.x86_64 #1 SMP F...
    netcdf_source:       baseYr_total_2000.nc
    title:               NCL: convert-NETCDF-to-netCDF
    Reference:           Gao, J., 2017: Downscaling Global Spatial Population...
    CDO:                 Climate Data Operators version 1.9.3 (http://mpimet....

In [38]:
GWL_tab.head()

,NEX-GDDP Models,Resolution,Experiment,warming_level,start_year,end_year
0,CNRM-CM5,22 km,RCP85,1.5,2015,2044
1,CSIRO-Mk3-6-0,22 km,RCP85,1.5,2018,2047
2,IPSL-CM5A-MR,22 km,RCP85,1.5,2002,2031
3,MIROC5,22 km,RCP85,1.5,2019,2048
4,MPI-ESM-LR,22 km,RCP85,1.5,2004,2033


In [39]:
def pop_process_gwl(start_year,end_year,target_file,model_name,scenario):
    
    print (start_year)
    print(end_year)
    dat=target_file.sel(time=slice(str(start_year)+"-01-01", str(end_year)+"-12-31")).mean('time')
    # Get output file_name
    outputfile='/mys3bucket/Pop_ssp5/Pop_SSP5_GWL_'+str(scenario)+'_'+str(model_name)+'_'+str(start_year)+'_'+str(end_year)+'.nc'                   
    # Create outpuut file
    print (outputfile)
    
    dat.to_netcdf(outputfile)

In [40]:
for i in range(GWL_tab.shape[0]):
    print(GWL_tab['NEX-GDDP Models'][i])
    #Get the right file to be processed
    target_file=pop
    
    # Get the start and the end year
    start_year=GWL_tab[' start_year'][i]
    end_year=GWL_tab[' end_year'][i]
    scenario=GWL_tab[' warming_level'][i]
    
    print ('Print target file')
    pop_process_gwl(start_year,end_year,target_file,GWL_tab['NEX-GDDP Models'][i],scenario)

CNRM-CM5
Print target file
2015
2044
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_1.5_CNRM-CM5_2015_2044.nc
CSIRO-Mk3-6-0
Print target file
2018
2047
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_1.5_CSIRO-Mk3-6-0_2018_2047.nc
IPSL-CM5A-MR
Print target file
2002
2031
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_1.5_IPSL-CM5A-MR_2002_2031.nc
MIROC5
Print target file
2019
2048
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_1.5_MIROC5_2019_2048.nc
MPI-ESM-LR
Print target file
2004
2033
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_1.5_MPI-ESM-LR_2004_2033.nc
NorESM1-M
Print target file
2019
2048
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_1.5_NorESM1-M_2019_2048.nc
GFDL-ESM2M
Print target file
2020
2049
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_1.5_GFDL-ESM2M_2020_2049.nc
CNRM-CM5
Print target file
2029
2058
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP5_GWL_2.0_CNRM-CM5_2029_2058.nc
CNRM-CM5
Print target file
2041
2070
/mys3bucket/Historical_cdd/Pop_ssp5/Pop_SSP

In [46]:
# xr.open_mfdataset('/mys3bucket/Historical_cdd/Pop_ssp5/test/Pop_SSP5_GWL_1.5*.nc')

In [16]:
## Compute population MODEL MEAN BY GLOBAL WARMING LEVELS

In [22]:
from cdo import Cdo
cdo = Cdo()

In [23]:
# ! conda create -n cdo -c conda-forge cdo python-cdo xarray netcdf4 matplotlib

In [24]:
## All RCP MODELS ENSMEAN GWL 1.0 under RCP8.5
cdo.ensmean(input='/mys3bucket/Pop_ssp5/Pop_SSP5_GWL_1.5*.nc',
             output='/mys3bucket/Pop_ssp5/MMM_Pop_SSP5_GWL_1.5_WA.nc')

cdo.timmean(input='/mys3bucket/Pop_ssp5/MMM_Pop_SSP5_GWL_1.5_WA.nc',
             output='/mys3bucket/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_1.5_WA.nc')

'/mys3bucket/Historical_cdd/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_1.5_WA.nc'

In [25]:
## All RCP MODELS ENSMEAN GWL 2.0 under RCP8.5
cdo.ensmean(input='/mys3bucket/Pop_ssp5/Pop_SSP5_GWL_2.0*.nc',
             output='/mys3bucket/Pop_ssp5/MMM_Pop_SSP5_GWL_2.0_WA.nc')

cdo.timmean(input='/mys3bucket/Pop_ssp5/MMM_Pop_SSP5_GWL_2.0_WA.nc',
             output='/mys3bucket/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_2.0_WA.nc')

'/mys3bucket/Historical_cdd/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_2.0_WA.nc'

In [26]:
## All RCP MODELS ENSMEAN GWL 2.5 under RCP8.5
cdo.ensmean(input='/mys3bucket/Pop_ssp5/Pop_SSP5_GWL_2.5*.nc',
             output='/mys3bucket/Historical_cdd/Pop_ssp5/MMM_Pop_SSP5_GWL_2.5_WA.nc')

cdo.timmean(input='/mys3bucket/Pop_ssp5/MMM_Pop_SSP5_GWL_2.5_WA.nc',
             output='/mys3bucket/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_2.5_WA.nc')

'/mys3bucket/Historical_cdd/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_2.5_WA.nc'

In [27]:
## All RCP MODELS ENSMEAN GWL 3.0 under RCP8.5
cdo.ensmean(input='/mys3bucket/Pop_ssp5/Pop_SSP5_GWL_3.0*.nc',
             output='/mys3bucket/Pop_ssp5/MMM_Pop_SSP5_GWL_3.0_WA.nc')

cdo.timmean(input='/mys3bucket/Pop_ssp5/MMM_Pop_SSP5_GWL_3.0_WA.nc',
             output='/mys3bucket/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_3.0_WA.nc')

'/mys3bucket/Historical_cdd/Pop_ssp5/CLIM_MMM_Pop_SSP5_GWL_3.0_WA.nc'